In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load
import time

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [ ]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

# LOAD DATA AND EXTRACT TESTSET

In [ ]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_wind_mm_cop.csv'

In [ ]:
raw_data: DataFrame = pd.read_csv(file_path)

In [ ]:
raw_data.head()

In [ ]:
raw_data['time'] = raw_data['time'].astype('datetime64[s]')

In [ ]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [ ]:
formatter: SorgeniaFormatter = config.make_data_formatter()

In [ ]:
data_csv_path: str = config.data_csv_path

SPLIT DATA

In [ ]:
train, valid, test = formatter.split_data(raw_data)

In [ ]:
test['time'].unique()[168]

PICK OBSERVED DATA

In [ ]:
forecast_time: str = '2020-08-13 22:00:00'

In [ ]:
obs: DataFrame = test[test['time']<= forecast_time]

In [ ]:
obs

In [ ]:
assert obs.shape[0] == 7*168

PICK KNOWN DATA

In [ ]:
known: DataFrame = test[(test['time'] > forecast_time) & (test['time'] <= pd.Timestamp(forecast_time) + pd.Timedelta(hours=12))]

In [ ]:
known

In [ ]:
assert known.shape[0] == 12*7
assert len(known['time'].unique()) == 12

CONCAT THE TWO DF

In [ ]:
inference_data: DataFrame = pd.concat([obs, known], axis=0, ignore_index=True)
inference_data: DataFrame = inference_data.sort_values(by=['plant_name_up','time'], ascending=True, ignore_index=True)

In [ ]:
inference_data

In [ ]:
assert inference_data.shape[0] == 7*168 + 7*12

In [ ]:
inference_data.columns

PREPARE INFRASTRUCTURE FOR TRAINING

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

In [ ]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
opt_manager.hyperparam_folder = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_mm+cop'
model_folder: str = opt_manager.hyperparam_folder
model_folder

# PREDICTION

In [ ]:
t0: float = time.perf_counter()
print("*** Computing Preds ***")
tf1.reset_default_graph()
with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
    tf1.keras.backend.set_session(sess)
    params: Dict = opt_manager.get_next_parameters()
    params['exp_name'] = 'sorgenia_wind'
    params['data_folder'] = os.path.abspath(os.path.join(data_csv_path, os.pardir))
    model = TemporalFusionTransformer(params, use_cudnn=False)
    params.pop('exp_name', None)
    params.pop('data_folder', None)
    # load model
    model.load(opt_manager.hyperparam_folder, use_keras_loadings=False)
    
#     output_map: Dict = model.predict(inference_data, return_targets=False)
#     p50_forecast: DataFrame = formatter.format_predictions(output_map.get("p50"))

t1: float = time.perf_counter()
print("Time elapsed ", t1-t0)

In [ ]:
# save model into hdf5
model.model.save(os.path.join(model.saved_models_folder, f"{model.exp_name}" + ".h5"))